[simpleT5](https://github.com/Shivanandroy/simpleT5)  
  
MIT License

Copyright (c) 2021 Shivanand Roy

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install simplet5

In [3]:
import json
import numpy as np
import pandas as pd

In [4]:
import os

DIR = "/content/drive/MyDrive/DACON_LG/data/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

In [5]:
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [6]:
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [7]:
train['total'] = train.title + ' ' + train.region + ' ' + train.context
test['total'] = test.title + ' ' + test.region + ' ' + test.context

In [8]:
# let's get a dataset
from sklearn.model_selection import train_test_split

In [9]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = train.rename(columns={"summary":"target_text", "total":"source_text"})
df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']

In [10]:
train_df, test_df = train_test_split(df, test_size=0.2)

In [11]:
print(len(train['context'].min()))
print(len(train['context'].max()))
print(len(train['summary'].min()))
print(len(train['summary'].max()))

367
1724
138
139


In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="mt5", model_name="google/mt5-small")
model.train(train_df=train_df,
            eval_df=test_df, 
            # source_max_token_len=1724, 
            target_max_token_len=139, 
            batch_size=8, max_epochs=25, early_stopping_patience_epochs = 2, use_gpu=True)

Global seed set to 42


Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

...

Validating: 0it [00:00, ?it/s]

Metric val_loss improved by 0.030 >= min_delta = 0.0. New best score: 0.978


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Metric val_loss improved by 0.028 >= min_delta = 0.0. New best score: 0.950


In [ ]:
text_to_summarize = test['total'].tolist()

In [ ]:
# let's load the trained model for inferencing:

result = []

model.load_model("mt5","/content/outputs/simplet5-epoch-24-train-loss-1.2206", use_gpu=True)

for i in range(len(text_to_summarize)):
  result.append(model.predict(text_to_summarize[i]))

result[:10]

In [ ]:
final_result = sum(result, [])

In [ ]:
final_result[:10]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/DACON_LG/data/sample_submission.csv')

In [ ]:
submission['summary'] = final_result

In [ ]:
submission.head()

In [ ]:
submission.to_csv('/content/drive/MyDrive/DACON_LG/data/mt5-trial.csv', index=False)